<!-- # CMPUT 200 Fall 2024  Ethics of Data Science and AI
 -->
# Assignment 1: Applying and Analyzing Privacy Techniques

***
- **Dataset**: titanic.csv
- **FIRST name**: Grace
- **LAST name**: Shin
- **Student ID**: 1788244

Leave blank if individual:
- **Collaborator names**: Karina Zhang
- **Collaborator student IDs**: 1766546
***

In this assignment you will apply basic privacy techniques to a dataset of your choosing. By the end of this assignment, you should be able to:
1. Understand and implement randomized response on binary data;
2. Calculate the sensititivity of a non-binary feature;
3. Add noise to a non-binary feature;
4. Compute aggregate statistics.

### Instructions
<!-- **Deadline.**  This assignment is due at ****.  Please check the syllabus for late submissions. -->
You are expected to write clear, detailed, and complete answers when analysing your data. Lack of this may result in point deductions.

**Reminder.** You must submit your own work. The collaboration policy for the assignments is Group Collaboration. You may work together in groups of up to 2.
 On Canvas we added a group set called "Assignment Group".  We have enabled self-signup for you to signup as a group.  If you don't select your group by Wednesday Sep 25th 11:59pm, we will assume you are working on your own.
 Under the group collaboration policy, besides working with your group member, you can discuss concepts with. your peers. However, the work must be by your own group, and all sources of information used including books, websites, students you talked to, must be cited in the submission. Please see the course FAQ document for details on this collaboration policy. We will adhere to current Faculty of Science guidelines on dealing with suspected cases of violation of academic integrity.

You must use this notebook to complete your assignment. You will execute the questions in the notebook. The questions might ask for a short answer in text form or for you to write and execute a piece of code. **Make sure you enter your answer in either case only in the cell provided.**  Do not use a different cell and do not create a new cell. Do not delete any test cells.  Creating new cells for your code and deleting test cells is not compatible with the auto-grading system we are using and thus your assignment will not be graded properly and you will lose marks for that question.

Your submitted notebook should run on our local installation.  So if you are importing packages not listed in the notebook or using local data files not included in the assignment package, make sure the notebook is self-contained with a requirements.txt file or cells in the notebook itself to install the extra packages.  If we cannot run your notebook, you will lose 50% of the marks, and any additional marks that may be lost due to wrong answers.

### Submission Instructions
When you are done, you will submit your work from the notebook. Make sure to save your notebook before running it, and then submit on Canvas the notebook file with your work completed.

**IMPORTANT: Name your file with your *Student ID number* and the assignment number (ex: 1234567_A1.ipynb). Failure to do so will result in a zero!**

In [5]:
# Run this cell to set up; Please don't change this cell.

import numpy as np
from numpy.random import default_rng
rng = default_rng()
import pandas as pd
from scipy.optimize import minimize

# These lines do some fancy plotting magic.
import matplotlib
# This is a magic function that renders the figure in the notebook, instead of displaying a dump of the figure object.
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

## Part 1: Data

**Question 1.1.** We will first load the data, carry out some cleaning and pre-processing, and inspect the data to understand what exploratory steps we will take. Name the DataFrame `df`.

In [6]:
# YOUR CODE HERE
# raise NotImplementedError()
import pandas as pd
df = pd.read_csv('titanic.csv')
print("Shape: ", df.shape)
df.head(5)

Shape:  (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**Question 1.2.** Describe your data and its purpose. Identify one variable that is binary or that could be classified into a binary feature. Identify another that is not a binary feature.

The data shows all the passengers that boarded the titanic and their personal information, such as sex, age, and class, and its purpose is to show the details of the passengers and if they survived. A feature that is binary is sex, where they are identified as either male or female. A feature that is not binary is their ticket number.

## Part 2: Data pre-processing

**Question 2.1.** If your data has missing values or empty rows, remove them in the cell below. If the feature that you chose above has to be classified into a binary feature, convert it. Finally, if your binary feature is categorical, convert it to numerial.

In [7]:
# YOUR CODE HERE
# raise NotImplementedError()
df.dropna(inplace=True)
df['Sex'] = df['Sex'].replace({'female': 0, 'male': 1})
print(df.head())
print(df.shape)

    PassengerId  Survived  Pclass  \
1             2         1       1   
3             4         1       1   
6             7         0       1   
10           11         1       3   
11           12         1       1   

                                                 Name  Sex   Age  SibSp  \
1   Cumings, Mrs. John Bradley (Florence Briggs Th...    0  38.0      1   
3        Futrelle, Mrs. Jacques Heath (Lily May Peel)    0  35.0      1   
6                             McCarthy, Mr. Timothy J    1  54.0      0   
10                    Sandstrom, Miss. Marguerite Rut    0   4.0      1   
11                           Bonnell, Miss. Elizabeth    0  58.0      0   

    Parch    Ticket     Fare Cabin Embarked  
1       0  PC 17599  71.2833   C85        C  
3       0    113803  53.1000  C123        S  
6       0     17463  51.8625   E46        S  
10      1   PP 9549  16.7000    G6        S  
11      0    113783  26.5500  C103        S  
(183, 12)


## Part 3: Randomized Response

Now let's implement a Randomized Response mechanism. First we have to identify what the query on our **binary** variable will be. Then we can create our own randomized mechanism.

**Question 3.1.** Write a query on your binary feature.

The query on my binary feature, Sex, will check if the individual is male. 

**Question 3.2.** Create your own randomized response mechanism for the query you defined above. You may NOT use the coin example from class, try to be creative!

Dice roll: 
if odd, say real answer
if even, roll again

second roll:
if odd, say 'yes'
if even, say 'no'

**Question 3.3.** Implement a function for your mechanism in 3.2. in the code cell below. The function should accept a value `0` or `1` and return the reported answer according to the randomized response above. Name your function `rand_resp`.

In [40]:
# YOUR CODE HERE
# raise NotImplementedError()
import numpy as np

# Function to implement the dice roll randomized response mechanism
def rand_resp(true_value):
    # Roll the first dice
    first_roll = np.random.randint(1, 7)
    
    # If the first dice roll is odd, tell the truth
    if first_roll % 2 != 0:
        return 1 if true_value == 1 else 0
    
    # If the first dice roll is even, roll again
    else:
        second_roll = np.random.randint(1, 7)
        # If the second dice roll is odd, say 'yes'
        if second_roll % 2 != 0:
            return 1
        # If the second dice roll is even, say 'no'
        else:
            return 0





**Question 3.4.** For each value in your dataframe's binary feature column, call your function. Store the results in a new column in df named `rrc1`.

In [46]:
# YOUR CODE HERE
# raise NotImplementedError()
df['rrc1'] = df['Sex'].apply(rand_resp)

# View the first few rows of the DataFrame to verify the new column
print(df.head())

    PassengerId  Survived  Pclass  \
1             2         1       1   
3             4         1       1   
6             7         0       1   
10           11         1       3   
11           12         1       1   

                                                 Name  Sex   Age  SibSp  \
1   Cumings, Mrs. John Bradley (Florence Briggs Th...    0  38.0      1   
3        Futrelle, Mrs. Jacques Heath (Lily May Peel)    0  35.0      1   
6                             McCarthy, Mr. Timothy J    1  54.0      0   
10                    Sandstrom, Miss. Marguerite Rut    0   4.0      1   
11                           Bonnell, Miss. Elizabeth    0  58.0      0   

    Parch    Ticket     Fare Cabin Embarked Is_male_randomized  rrc1  
1       0  PC 17599  71.2833   C85        C                 no     0  
3       0    113803  53.1000  C123        S                 no     0  
6       0     17463  51.8625   E46        S                 no     0  
10      1   PP 9549  16.7000    G6        

**Question 3.5.** Now get the **estimate** for the true number of people who answered `1`.  Write this result into the variable `count_est_true_yes`.  Given the number of reported `1`'s, we know how to estimate the proportion or number of true `1`'s.

Calculate the true number of people who answered `1` (the true answer in the data we imported) and write it into the variable `count_true_yes`.

In [47]:
# YOUR CODE HERE
# raise NotImplementedError()
count_true_yes = df['Sex'].sum()
reported_yes_count = (df['rrc1'] == 1).sum()
p_truth = 3/6
random_guessing_proportion = 0.5
n = len(df)
count_est_true_yes = (reported_yes_count - (random_guessing_proportion * (n - reported_yes_count))) / p_truth
print("Estimated true yes count: ", count_est_true_yes)
print("True yes count: ", count_true_yes)

Estimated true yes count:  87.0
True yes count:  95


**Question 3.6.** Comment on your results from above. What can you say about the privacy-accuracy tradeoff of your randomized response mechanism?

The estimated true yes count and the true yes count in the results are pretty close to each other. The privacy-accuracy tradeoff of my randomized response mechanism gives more accuracy due to smaller randomization.

**Question 3.7.** We learned in class that data analysts are still able to obtain aggregate statistics from the results of a randomized response survey. Using the column you made above, `rrc1`, to calculate the mean and median for the estimated true responses. Do the same for your true responses. Name your answers `mean_est_true_yes`, `mean_true_yes`, `median_est_true_yes`, and `median_true_yes` respectively.

In [50]:
# YOUR CODE HERE
# raise NotImplementedError()
import numpy as np
mean_true_yes = df['Sex'].mean()
median_true_yes = df['Sex'].median()


mean_est_true_yes = df['rrc1'].mean()
median_est_true_yes = df['rrc1'].median()

print("Mean: ", mean_est_true_yes, mean_true_yes)
print("Median: ", median_est_true_yes, median_true_yes)

Mean:  0.4918032786885246 0.5191256830601093
Median:  0.0 1.0


**Question 3.8.** Comment on your results from above. Are the results from your mechanism useable? What can you say about the privacy when it comes to randomized response? Comment on the distributions of your data.

The mean of the estimated true responses and the real true responses are very similar to each other, meaning that the mean of the estimated true responses approximates the real proportion of males in the population. The randomness is what causes the slight deviation. The median of the estimated true responses may show more variability, as the randomization would cause the middle value to differ and shift according to the dice rolls. The results from my mechanism are useable, as there is sufficient randomization while still maintaining some accuracy through the mean. The privacy of the randomized response is strong, as the randomization still protects the individual responses, but comes at the cost of some accuracy, as with the median values. The distribution of the data for the estimated true responses will be noisier than the true distribution, which is the reason for the deviations in the mean and median values. 

## Part 4: Adding Noise

We are going to use the non-binary feature that you chose earlier and add noise to it. To do this, we apply the same steps that we would for differential privacy: $f(D) + Z$.

**Question 4.1.** Suppose the function we wish to query is **count**. What would the global sensitivity, $S(f)$, be? Explain why.

The global sensitivitiy for a count query would be 1, since the most that changes when a single individual is removed or added from the dataset would be 1.

**Question 4.2.** In the cell below, write a function that adds Laplace noise to a value given the sensitivity and the privacy parameter, epsilon. Name your function `add_laplace_noise`.

In [51]:
# YOUR CODE HERE
# raise NotImplementedError()
def add_laplace_noise(value, sensitivity, epsilon):
    # Scale parameter for the Laplace distribution
    scale = sensitivity / epsilon
    
    # Generate Laplace noise
    noise = np.random.laplace(0, scale)
    
    # Add the noise to the original value
    noisy_value = value + noise
    
    return noisy_value

# Example usage:
# Let's say we have a value 10, sensitivity of 1, and epsilon of 0.5
noisy_value = add_laplace_noise(10, 1, 0.5)
print("Noisy value:", noisy_value)

Noisy value: 11.043388131189829


Since our query is on count, we'll have to obtain the count of each unique value of our feature.

**Question 4.3.** Define a variable called that holds the count of each unique value of your feature. *Hint: there's a method that does this for us!*

In [54]:
# YOUR CODE HERE
# raise NotImplementedError()
value_count = df['Sex'].value_counts()

**Question 4.4.** Before we add noise, let's calculate some stats from your variable from 4.3. Calculate the mean, median, and count and name them `mean_count`, `median_count`, and `total_count` respectively.

In [57]:
# # YOUR CODE HERE
# raise NotImplementedError()
# Get the counts of each unique value from the 'Sex_binary' feature
value_counts = df['Sex'].value_counts()

# Calculate the mean, median, and total count
mean_count = value_counts.mean()         # Mean of the counts
median_count = value_counts.median()     # Median of the counts
total_count = value_counts.sum()         # Total number of individuals

# Output the results

print(f"Mean of true counts: {mean_count}")
print(f"Median of true counts: {median_count}")
print(f"Total count of true records: {total_count}")

Mean of true counts: 91.5
Median of true counts: 91.5
Total count of true records: 183


**Question 4.5.** Now we can start adding noise. Use your Laplace function and your variable from 4.3. to calculate a noisy representation of each value in your feature. Set your value of epsilon to 1 for now.

In [59]:
# YOUR CODE HERE
# raise NotImplementedError()
epsilon = 1
sensitivity = 1

# Use the Laplace noise function to create a noisy version of the counts from 4.3
noisy_value_counts = value_counts.apply(lambda count: add_laplace_noise(count, sensitivity, epsilon))

# Output the noisy counts
print("Original counts:\n", value_counts)
print("Noisy counts:\n", noisy_value_counts)

Original counts:
 Sex
1    95
0    88
Name: count, dtype: int64
Noisy counts:
 Sex
1    95.622934
0    87.831890
Name: count, dtype: float64


**Question 4.6.** Now calculate the stats for your noisy values. Calculate the mean, median, and count and name them `mean_noisy_count`, `median_noisy_count`, and `total_noisy_count` respectively.

In [61]:
# YOUR CODE HERE
# raise NotImplementedError()
# Calculate the mean, median, and total count for the noisy values
mean_noisy_count = noisy_value_counts.mean()         # Mean of the noisy counts
median_noisy_count = noisy_value_counts.median()     # Median of the noisy counts
total_noisy_count = noisy_value_counts.sum()         # Total of the noisy counts
print(f"Noisy mean of true counts: {mean_noisy_count}")
print(f"Noisy median of true counts: {median_noisy_count}")
print(f"Total noisy count of true records: {total_noisy_count}")

Noisy mean of true counts: 91.72741170777385
Noisy median of true counts: 91.72741170777385
Total noisy count of true records: 183.4548234155477


**Question 4.7.** Comment on the differences in aggregate statistics between the original and the noisy values. What can you say about the utility and privacy?  

The difference in aggregate statistics between the original and the noisy values are that the noisy mean and medians are slightly higher than the original values. There is also more accuracy in the numbers after the decimal points for the noisy values. As the noisy values still remain close to the original values, the utility is high. The privacy is high as the Laplace noise guarantees strong privacy by making sure that individual data does not have a big impact on the overall values. The epsilon value of 1 shows a balance between utility and privacy, providing a good tradeoff between the two.

**Question 4.8.** Go back to question 4.5. and change the value of epsilon. Repeat this until you notice a pattern in your aggregate statistics. What happens as epsilon changes? What happens to the privacy and the accuracy?

As epsilon gets smaller, the privacy is stronger, and the amount of noise added to the results is significant. This causes the accuracy of the results to decrease. As epsilon gets larger, the accuracy of the results increase, but gives a weaker privacy guarantee due to less noise being added to the results. This gives a greater risk of revealing individual data points. 

# Rubric

| Question | Points|
|----------|----------|
| 1.1.   | 5   |
| 1.2.    | 10   |
| 2.1.    | 5   |
| 3.1.   | 5   |
| 3.2.    | 10  |
| 3.3.  | 10   |
| 3.4.    | 2   |
| 3.5.   | 8   |
| 3.6.   | 5   |
| 3.7.   | 6   |
| 3.8.   | 8   |
| 4.1.   | 2   |
| 4.2.    | 10  |
| 4.3.  | 5   |
| 4.4.    | 3   |
| 4.5.   | 5   |
| 4.6.   | 3   |
| 4.7.   | 5   |
| 4.8.   | 8   |
| Total  | 115   |


